## 10. 미니 프로젝트

In [1]:
import cv2  
import numpy as np
CARD = "images/card2.jpg"

In [3]:
import cv2
import numpy as np

# 이미지를 읽고 원본 이미지를 복사하여 저장
img = cv2.imread("images/card2.jpg")  # card2.jpg 파일 읽기
original_img = img.copy()  # 원본 이미지 복사 (나중에 변형 전 이미지를 사용하기 위해)

point_list = []  # 마우스 클릭으로 선택한 점을 저장할 리스트
current_mouse_pos = None  # 현재 마우스 위치를 저장 (임시 선을 그리기 위해)

# 마우스 이벤트 처리 함수
def on_mouse(event, x, y, flags, _):
    """
    마우스 이벤트 핸들러
    - 좌클릭으로 점을 추가
    - 마우스 이동 시 임시 선 그리기
    """
    global point_list, img, current_mouse_pos

    if event == cv2.EVENT_LBUTTONDOWN:  # 좌클릭 이벤트
        if len(point_list) < 4:  # 점이 4개 미만일 때만 추가
            point_list.append((x, y))  # 클릭한 점 추가
            lines_img()  # 현재까지의 점과 선 업데이트

        if len(point_list) == 4:  # 점이 4개가 되면 문서를 변환
            warp_document()

    elif event == cv2.EVENT_MOUSEMOVE:  # 마우스 이동 이벤트
        current_mouse_pos = (x, y)  # 현재 마우스 위치 저장
        if len(point_list) > 0 and len(point_list) < 4:  # 점이 1개 이상 4개 미만일 때
            draw_temp_lines()  # 임시 선을 그리기

# 클릭된 점과 선을 그리는 함수
def lines_img():
    """
    마우스 클릭으로 선택된 점들과 연결된 선을 이미지에 표시
    """
    global img, point_list
    img = original_img.copy()  # 원본 이미지로 초기화

    # 각 점에 원을 그림
    for point in point_list:
        cv2.circle(img, point, 5, (0, 255, 255), cv2.FILLED)  # 점 표시

    # 점들을 순서대로 연결하여 선을 그림
    for i in range(1, len(point_list)):
        cv2.line(img, point_list[i - 1], point_list[i], (0, 255, 255), 2)

    # 점이 4개일 경우, 마지막 점과 첫 번째 점을 연결
    if len(point_list) == 4:
        cv2.line(img, point_list[-1], point_list[0], (0, 255, 255), 2)

    # 이미지를 업데이트하여 표시
    cv2.imshow("img", img)

# 마우스 이동 시 임시 선을 그리는 함수
def draw_temp_lines():
    """
    현재 마우스 위치를 기준으로 임시 선을 표시
    """
    global img, current_mouse_pos
    temp_img = img.copy()  # 현재 이미지를 복사

    # 선택된 점과 연결된 선을 그림
    for point in point_list:
        cv2.circle(temp_img, point, 5, (0, 255, 255), cv2.FILLED)  # 점 표시
    for i in range(1, len(point_list)):
        cv2.line(temp_img, point_list[i - 1], point_list[i], (0, 255, 255), 2)

    # 마지막 점과 현재 마우스 위치를 연결하는 임시 선을 그림
    if len(point_list) > 0:
        cv2.line(temp_img, point_list[-1], current_mouse_pos, (0, 255, 255), 1)

    # 업데이트된 이미지를 표시
    cv2.imshow("img", temp_img)

# 선택된 점을 기준으로 원근 변환 수행
def warp_document():
    """
    선택한 4개의 점을 기준으로 이미지를 원근 변환
    """
    width,height = 600,350
    global point_list
    pts = np.array(point_list,dtype=np.float32)  # 선택된 점들을 float32로 변환
    dst = np.array([[0, 0], [width, 0], [width, height], [0, height]], dtype=np.float32)  # 목적 좌표 (사각형)
    mat = cv2.getPerspectiveTransform(pts, dst)  # 원근 변환 행렬 계산
    result = cv2.warpPerspective(original_img, mat, (width, height))  # 원근 변환 적용

    # 변환된 이미지를 표시
    cv2.imshow("result", result)

# 초기 이미지 창 열기
cv2.imshow("img", img)  # 이미지를 표시
cv2.setMouseCallback("img", on_mouse)  # 마우스 이벤트 핸들러 등록
cv2.waitKey(0)  # 키 입력 대기
cv2.destroyAllWindows()  # 모든 창 닫기


In [22]:
import cv2
import numpy as np

img = cv2.imread("images/card2.jpg")
point_list = []

# 최종 결과를 출력하는 함수입니다
def show_result():
  width, height = 600, 350
  src = np.array(point_list, dtype=np.float32)
  # src = np.float32(point_list)
  target = np.array([[0,0], [width, 0], [width, height], [0, height]], dtype=np.float32)
  mat = cv2.getPerspectiveTransform(src, target)
  result = cv2.warpPerspective(img, mat, (width, height))
  cv2.imshow("result", result)

# 마우스 이벤트와 함께 실행되는 콜백함수입니다.
drawing = False
def on_mouse(event, x, y, flags, _):
  global drawing
  coppied = img.copy()
  if event == cv2.EVENT_LBUTTONDOWN:
    drawing = True
    point_list.append((x,y))

  if drawing:
    prev_point = None
    for point in point_list:
      cv2.circle(coppied, point, 10, (0,0,255), cv2.FILLED)
      if prev_point:
        cv2.line(coppied, prev_point, point, (0,0,255), 5)
      prev_point = point

    next_point = (x,y)
    if len(point_list) == 4:
      next_point = point_list[0]
      show_result()
    cv2.line(coppied, point_list[-1], next_point, (0,0,255), 5)

  cv2.imshow(name, coppied)

# 기본적인 코드 실행입니다.
name = "Scanner"
cv2.namedWindow(name)
cv2.setMouseCallback(name, on_mouse)
cv2.imshow(name, img)
cv2.waitKey(0)
cv2.destroyAllWindows()